In [2]:
import os
import sys
import numpy as np
sys.path.append('../')
import scipy.stats as stats
from utils import get_distance, haversineDistance
from placecell import PlaceNetwork, loadNetwork

In [3]:
PATH_DIR = 'paths'

In [4]:
def two_sample_t_test(data1, data2, alpha=0.05):
    t_statistic, p_value = stats.ttest_ind(data1, data2)
    degrees_of_freedom = len(data1) + len(data2) - 2
    critical_value = stats.t.ppf(1 - alpha / 2, degrees_of_freedom)
    reject_null = abs(t_statistic) > critical_value

    result = {
        't_statistic': t_statistic,
        'p_value': p_value,
        'degrees_of_freedom': degrees_of_freedom,
        'critical_value': critical_value,
        'reject_null': reject_null
    }

    return result

def wilcoxon_ranksum_test(data1, data2):
    statistic, p_value = stats.ranksums(data1, data2)

    result = {
        'statistic': statistic,
        'p_value': p_value,
    }
    return result

In [5]:
def stringToTuple(input_string):
    try:
        tuples_list = eval(input_string)
        
        if isinstance(tuples_list, list) and all(isinstance(t, tuple) and len(t) == 2 for t in tuples_list):
            return tuples_list
        else:
            raise ValueError("Invalid input format. Must be a list of tuples.")
    
    except Exception as e:
        raise ValueError("Error converting string to tuples: {}".format(str(e)))

def evalCost(input_string):
    try:
        # Using eval to parse the string into a Python expression
        list_of_lists = eval(input_string)
        
        # Check if the result is a list of lists
        if isinstance(list_of_lists, list) and all(isinstance(sublist, list) for sublist in list_of_lists):
            # Check if each element in the sublists is a float
            flattened_list = [elem for sublist in list_of_lists for elem in sublist]
            return list_of_lists
        else:
            raise ValueError("Invalid input format. Must be a list of lists.")
    
    except Exception as e:
        return None
        raise ValueError("Error converting string to list of lists: {}".format(str(e)))

In [6]:
dir_list = os.listdir(PATH_DIR)

paths = {'RRT' : {}, 'spike' : {}}
start_ends = []

for file in dir_list:
    if os.path.isdir(os.path.join(PATH_DIR, file)):
        continue
    comps = file[0:-4].split("_")
    
    pathtype = comps[0]
    
    start = comps[1].split("-")
    start = (int(start[0]), int(start[1]))

    end = comps[2].split("-")
    end = (int(end[0]), int(end[1]))

    point = ((start, end))

    with open(os.path.join(PATH_DIR, file)) as f:
        path = stringToTuple(f.readline())
        costs = evalCost(f.readline())
        if costs != None:
            paths[pathtype][point] = {'trajectory' : None, 'cost' : None}
            paths[pathtype][point]['trajectory'] = path
            paths[pathtype][point]['cost'] = costs

    if point not in start_ends and costs != None:
        start_ends.append(point)

In [7]:
def calculateCost(data):
    current_total = 0
    current_time = 0

    obstacle_total = 0
    obstacle_time = 0

    slope_total = 0
    slope_time = 0

    for wp in data:
        current_total += wp[1]
        current_time += wp[2]
        obstacle_total += wp[3]
        obstacle_time += wp[4]
        slope_total += wp[5]
        slope_time += wp[6]

    return [current_total, current_time, obstacle_total, obstacle_time, slope_total, slope_time]

In [8]:
rrts = []
swps = []

rrt_plens = []
swp_plens = []

for pts in start_ends:
    rrt_costs = paths['RRT'][pts]['cost']
    swp_costs = paths['spike'][pts]['cost']

    rrts.append(calculateCost(rrt_costs))
    swps.append(calculateCost(swp_costs))

    rrt_plens.append(len(paths['RRT'][pts]['trajectory']))
    swp_plens.append(len(paths['spike'][pts]['trajectory']))

if len(rrts) != len(swps):
    print("Number of data not the same")

In [9]:
n = PlaceNetwork()
n.initAldritch(numcosts=6)
n.initConnections()

In [10]:
rrt_dists = []
swp_dists = []

for pts in start_ends:
    rrt = paths['RRT'][pts]['trajectory']
    swp = paths['spike'][pts]['trajectory']

    rrt_d = 0
    swp_d = 0

    for i in range(len(rrt) - 1):
        d1 = n.cells[n.points[rrt[i]]].origin
        d2 = n.cells[n.points[rrt[i+1]]].origin

        rrt_d += haversineDistance(d1[0], d1[1], d2[0], d2[1])

    rrt_dists.append(rrt_d)

    for i in range(len(swp) - 1):
        d1 = n.cells[n.points[swp[i]]].origin
        d2 = n.cells[n.points[swp[i+1]]].origin
    
        swp_d += haversineDistance(d1[0], d1[1], d2[0], d2[1])

    swp_dists.append(swp_d)
        

In [13]:
distance = two_sample_t_test(swp_dists, rrt_dists)
print("Spiking wavefront versus rrt")
print(f"Mean swp distance: {np.mean(swp_dists)}")
print(f"Mean rrt distance: {np.mean(rrt_dists)}")
print(f"p-value: {distance['p_value']}")

Spiking wavefront versus rrt
Mean swp distance: 46.60400973848205
Mean rrt distance: 60.24742640545722
p-value: 0.01125063503687932


In [12]:
swp_curr = [swps[i][1] / swps[i][0] for i in range(len(swps))]
rrt_curr = [rrts[i][1] / rrts[i][0] for i in range(len(rrts))]

swp_obs = [swps[i][2] for i in range(len(swps))]
rrt_obs = [rrts[i][2] for i in range(len(rrts))]

swp_slope = [swps[i][4] for i in range(len(swps))]
rrt_slope = [rrts[i][4] for i in range(len(rrts))]

curr_result = two_sample_t_test(swp_curr, rrt_curr)
print(f"Mean swp current: {np.mean(swp_curr)}")
print(f"Mean rrt current: {np.mean(rrt_curr)}")
print(f"p-value: {curr_result['p_value']}")

print("___________________________________________")

obs_result = two_sample_t_test(swp_obs, rrt_obs)
print(f"Mean swp current: {np.mean(swp_obs)}")
print(f"Mean rrt current: {np.mean(rrt_obs)}")
print(f"p-value: {obs_result['p_value']}")

print("___________________________________________")

slope_result = two_sample_t_test(swp_slope, rrt_slope)
print(f"Mean swp current: {np.mean(swp_slope)}")
print(f"Mean rrt current: {np.mean(rrt_slope)}")
print(f"p-value: {slope_result['p_value']}")

Mean swp current: 0.3032510635033979
Mean rrt current: 0.3099980347605631
p-value: 0.7280681460292979
___________________________________________
Mean swp current: 29.16
Mean rrt current: 67.8
p-value: 0.0065869009404392895
___________________________________________
Mean swp current: 1593.0148261077345
Mean rrt current: 1789.3336772593498
p-value: 0.3464791692955913
